In [1]:
# Importing libraries...
import time
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

from GoPiGo3 import GoPiGo3

# Init client
client = RemoteAPIClient()  # Client object
sim = client.getObject("sim")  # Simulation object

In [2]:
def init_population(population_size, genome_lenght):
    return np.random.uniform(0.0, 3.0, (population_size, genome_lenght))


def crossover(genome_x, genome_y, crossover_rate):
    if np.random.rand() < crossover_rate:
        c = np.random.randint(0, len(genome_x) - 1)

        return np.concatenate((genome_x[:c], genome_y[c:])), np.concatenate((genome_y[:c], genome_x[c:]))
    else:
        return genome_x, genome_y


def mutate(genome, mutation_rate):
    new_genome = genome * np.random.uniform(1.0 - mutation_rate, 1.0 + mutation_rate, len(genome)) # It can drop or rise by 25%
    return new_genome


def get_fitness(metrics, weights):
    return np.dot(metrics, weights)


def pick_fittest(population, fitness_values):
    return population[np.argsort(fitness_values)][-2:] # Pick fittest parents

In [3]:
# Resetting Individuals
def reset_individuals(population_size):
    while True:
        try:
            sim.removeModel(sim.getObject(f"/Individual[1]"))
        except:
            break

    base_handle = sim.getObject("/Individual")
    for _ in range(population_size - 1):
        sim.copyPasteObjects([base_handle], 1)

In [ ]:
# Simulation batch
simulation_time = 5.0
target_lower_bound = 0.1
target_higher_bound = 1.0

generations = 50
population_size = 10
crossover_rate = 0.10
mutation_rate = 0.10
weights = np.array([-40.0, -1.0, -20.0, -50.0, -50.0])

population = np.repeat([[3.0, 0.001, 0.01, 1.0, 0.001, 0.01]], population_size, axis=0)

fitness_values = None

reset_individuals(population_size)

for generation, (x, y) in enumerate(np.random.uniform(-0.5, 0.5, (generations, 2))):
    print(f"Generation {generation}: Target at x={x:.2f} ; y={y:.2f} m")

    sim.setObjectPosition(
        sim.getObject('/Target'),
        [x, y, 0.05]
    )

    robot_population = [
        GoPiGo3(
            simulation_object=sim,
            handle_name=f"/Individual[{i}]/GoPiGo3",
            target_handle_name="/Target",
            genome=genome
        )
        
        for i, genome in enumerate(population)
    ]

    try:
        sim.setStepping(True)  # Trigger simulation step manually

        # Start simulation
        sim.startSimulation()

        while sim.getSimulationTime() < simulation_time:
            for robot in robot_population:
                
                robot.sense_and_actuate()

            sim.step() # Call for next step

        # Stop simulation
        sim.stopSimulation()
        time.sleep(1.0) # Wait for simulation to stop

    except:
        pass
    
    # Get fitnes of every individual of the population
    fitness_values = [get_fitness(robot.get_metrics(), weights) for robot in robot_population]

    for robot, fitness_value in zip(robot_population, fitness_values):
        metrics = robot.get_metrics()
        
        print(f"Not converged: {robot.not_converged}") 
        print(f"Convergence time: {robot.convergence_time:.2f}") 
        print(f"Target distance: {robot.target_distance:.2f}") 
        print(f"Max roll: {robot.max_roll:.3f}") 
        print(f"Max pitch: {robot.max_pitch:.3f}") 
        print()
        print(robot.genome)
        print()
        print(f"Fitness: {fitness_value:.2f}", '\n\n')

    # Pick fittest and have them make them have offspring
    parent_x, parent_y = pick_fittest(population, fitness_values)

    # If it is on last generation, do not crossover and mutate
    if generation >= generations - 1:
        break

    # Make a new population based on variants of the most adapted individuals 
    new_population = []
    offspring_x, offspring_y = crossover(parent_x, parent_y, crossover_rate)

    for _ in range(population_size // 2):
        new_population.append(mutate(offspring_x, mutation_rate))
        new_population.append(mutate(offspring_y, mutation_rate))

    # Update population
    population = np.array(new_population)

Generation 0: Target at x=0.05 ; y=0.49 m
Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

Fitness: -6.20 


Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

Fitness: -6.20 


Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

Fitness: -6.20 


Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

Fitness: -6.21 


Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

Fitness: -6.22 


Not converged: False
Convergence time: 4.20
Target distance: 0.01
Max roll: 0.018
Max pitch: 0.018

[3.e+00 1.e-03 1.e-02 1.e+00 1.e-03 1.e-02]

In [10]:
# Create the controller
genome = pick_fittest(population, fitness_values)[-1] # Pick fittest genome

print(genome)

reset_individuals(1)

robot = GoPiGo3(
    simulation_object=sim,
    handle_name="/Individual/GoPiGo3",
    target_handle_name="/Target",
    genome=genome
)

try:
    sim.setStepping(True)  # Trigger simulation step manually

    # Start simulation
    sim.startSimulation()
    
    while not sim.getSimulationStopping():
        robot.sense_and_actuate()
        sim.step()

    # Stop simulation
    sim.stopSimulation()

except:
    pass

[5.15442269 6.36117677 6.65774828 1.36165467 0.07150544 0.04885606]
